In [1]:
import plotly.plotly as py
from plotly import tools
import numpy as np
from plotly.graph_objs import *
import plotly.graph_objs as go
import math


In [2]:

def walsh_percent_effect_remaining_at_time(time, action_duration):
#     if t <= 0:
#         return 1
#     elif t >= action_duration:
#         return 0
    
    nearest_modeled_duration = None
   
    if action_duration < 3:
        nearest_modeled_duration = 3
    elif action_duration > 6:
        nearest_modeled_duration = 6
    else: 
        nearest_modeled_duration = round(action_duration)
    
    minutes = (time*60) * nearest_modeled_duration / action_duration
    
    if nearest_modeled_duration == 3:
        return -3.2030e-9 * pow(minutes, 4) + 1.354e-6 * pow(minutes, 3) - 1.759e-4 * pow(minutes, 2) + 9.255e-4 * minutes + 0.99951
    elif nearest_modeled_duration == 4:
        return -3.310e-10 * pow(minutes, 4) + 2.530e-7 * pow(minutes, 3) - 5.510e-5 * pow(minutes, 2) - 9.086e-4 * minutes + 0.99950
    elif nearest_modeled_duration == 5:
        return -2.950e-10 * pow(minutes, 4) + 2.320e-7 * pow(minutes, 3) - 5.550e-5 * pow(minutes, 2) + 4.490e-4 * minutes + 0.99300
    elif nearest_modeled_duration == 6:
        return -1.493e-10 * pow(minutes, 4) + 1.413e-7 * pow(minutes, 3) - 4.095e-5 * pow(minutes, 2) + 6.365e-4 * minutes + 0.99700
    else:
        raise Exception('Invalid!')
    

In [6]:
# New decay curve
#
tau = 0.9
old_dia = 4
new_dia = 6

new_x = np.linspace(0,new_dia,num=new_dia*100)
new_decay = [tau * (tau - (t + tau) * math.exp(-t/tau)) for t in new_x]
new_decay = 1 - np.array(new_decay) / pow(tau,2)
exp_model_name = "exp model (tau=%d)" % tau
new_trace = Scatter(x=new_x, y=new_decay, name=exp_model_name + " decay")

walsh_x = np.linspace(0,old_dia,old_dia*100)
walsh_decay = np.array([walsh_percent_effect_remaining_at_time(t,4) for t in walsh_x])
walsh_model_name = "Walsh %d hour" % old_dia
walsh_trace = Scatter(x=walsh_x, y=walsh_decay, name=walsh_model_name + " decay")

data = Data([new_trace,walsh_trace])

fig = tools.make_subplots(rows=2, cols=1, subplot_titles=('Insulin On Board Decay', 'Insulin Activity'))

fig['layout'].update(height=600, width=600)

fig.append_trace(new_trace, 1, 1)
fig.append_trace(walsh_trace, 1, 1)


new_activity = np.gradient(new_decay)
new_trace = Scatter(x=new_x, y=-new_activity, name=exp_model_name + " activity")

# Scale activity so that total activity is equal 
new_total = new_activity.sum()
walsh_activity = np.gradient(walsh_decay)
walsh_total = walsh_activity.sum()
print walsh_total
print new_total
scale = new_total / walsh_total
print walsh_total * scale

walsh_trace = Scatter(x=walsh_x * scale, y=-walsh_activity, name=walsh_model_name + " activity")

fig.append_trace(new_trace, 2, 1)
fig.append_trace(walsh_trace, 2, 1)

py.iplot(fig, filename='make-subplots-multiple-with-titles')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]

-0.993399874177
-0.990321316659
-0.990321316659
